In [ ]:
import numpy as np
import pennylane as qml
import torch

from q_alchemy.pennylane_integration import AmplitudeEmbedding, OptParams
from sklearn.datasets import make_moons

# Sample data
X, _ = make_moons(n_samples=5, noise=0.1)
X = X / np.linalg.norm(X, axis=1, keepdims=True)  # Normalize each row for amplitude embedding

# Create PennyLane device
dev = qml.device("qiskit.aer", wires=1)

@qml.qnode(dev, interface="torch")
def circuit(x):
    AmplitudeEmbedding(
        x,
        wires=[0],
        use_research_function="baa_tucker_initialize",
        opt_params=OptParams(
            max_fidelity_loss=0.0,
            basis_gates=["id", "rx", "ry", "rz", "cx"],
            api_key="<your api key>"
        )
    )
    return qml.expval(qml.PauliZ(0))

# Run the circuit on a batch of inputs
X_tensor = torch.tensor(X, dtype=torch.float32)
results = torch.stack([circuit(x) for x in X_tensor])
print(results)